# GPT Actions - Snowflake ミドルウェア
## はじめに

このページは、特定のアプリケーション向けのGPT Actionを構築する開発者向けの手順とガイドを提供します。進める前に、まず以下の情報をよく理解しておいてください：

* [GPT Actionsの紹介](https://platform.openai.com/docs/actions)
* [GPT Actions Libraryの紹介](https://platform.openai.com/docs/actions/actions-library)
* [GPT Actionを一から構築する例](https://platform.openai.com/docs/actions/getting-started)

このガイドでは、[データ分析](https://help.openai.com/en/articles/8437071-data-analysis-with-chatgpt)で使用するためのSQLクエリをChatGPTに返すことを目的として、ChatGPTをSnowflake Data Warehouseに接続する方法の詳細を説明します。GPTには、ミドルウェア（Azure functionなど）とインターフェースするアクションが必要です。これにより、アクションがSnowflakeからのレスポンスをPythonノートブック環境で使用するために適切にフォーマットできます。データは[ファイルとして返される](https://platform.openai.com/docs/actions/sending-files/returning-files)必要があるため、ミドルウェア関数はSQLレスポンスを10MB未満のCSV/Excelファイルに変換する必要があります。

このドキュメントでは、ミドルウェア関数GPTアクションについて概説します。ミドルウェア関数自体の設定については、[GPT Actions library (Middleware) - Azure Functions](https://cookbook.openai.com/examples/chatgpt/gpt_actions_library/gpt_middleware_azure_function)を参照してください。このSnowflakeミドルウェアアクションを、Snowflakeへの直接アクションと組み合わせることで、SQLクエリを実行前に形成およびテストできるGPTを有効にすることができます。

### 価値 + ビジネス活用事例

既存のSnowflakeのお客様は、これらのガイドラインを活用して、データウェアハウスからデータをクエリし、そのデータをData Analysis Python環境に読み込んで、さらなる洞察を得ることができます。これにより、データセットの可視化、パターンや異常の特定、データクレンジング目的でのギャップの特定など、ChatGPTを活用した分析が可能になります。このGPTは、比較的小規模なデータセットからビジネス上の意思決定を推進したり、BIツールで包括的なデータセットを探索する際に、AIを通じてデータのサブセットを調査して仮説を生成したりするために使用できます。これにより、時間とコストを節約しながら、これまで見えなかったパターンを特定することができます。

## アプリケーション情報

### アプリケーションキーリンク

開始する前に、SnowflakeとAzureからのこれらのリンクをチェックしてください：

**Snowflake Action**

* アプリケーションWebサイト: [https://app.snowflake.com/](https://app.snowflake.com/)
* アプリケーションPython Connectorドキュメント: [https://docs.snowflake.com/en/developer-guide/python-connector/python-connector-connect](https://docs.snowflake.com/en/developer-guide/python-connector/python-connector-connect)

**Azure Function**

* アプリケーションWebサイト: [https://learn.microsoft.com/en-us/azure/azure-functions/](https://learn.microsoft.com/en-us/azure/azure-functions/)
* アプリケーションAPIドキュメント: [https://learn.microsoft.com/en-us/azure/azure-functions/functions-reference/](https://learn.microsoft.com/en-us/azure/azure-functions/functions-reference/)

### アプリケーションの前提条件

開始する前に、アプリケーション環境で以下の手順を実行していることを確認してください：

* Snowflake Data Warehouseをプロビジョニングする
* ChatGPT経由でSnowflakeに認証するユーザーが、必要なロールでデータベース、スキーマ、テーブルにアクセスできることを確認する

さらに、Azure Function Appでアプリケーションを作成する前に、ユーザー認証を処理する方法が必要です。SnowflakeのExternal OAuthセキュリティ統合とリンクできるOAuth App RegistrationをAzure Entra IDに設定する必要があります。SnowflakeのExternal OAuthセキュリティ統合により、外部システムがアクセストークンを発行し、Snowflakeがアクセスレベルの決定に使用できるようになります。この場合、その外部トークンプロバイダーはAzure Entra IDです。ChatGPTはSnowflakeではなくAzureに接続するため、GPTユーザーのOAuthトークンは、Entra IDの対応するユーザーに関連付けられたAzureによってプロビジョニングされます。したがって、SnowflakeのユーザーをAzureの対応するユーザーにマッピングする方法が必要になります。

Azure側とSnowflake側の両方に必要なすべての手順を以下に示します。

## Azure Entra IDでOAuthリソースを設定する

新しいApp Registrationを設定し、使用するAzureで必要なSnowflake Scopesを構成し、SnowflakeとChatGPTの両方で必要となるすべてのOAuth設定パラメータを取得します。このセクションはすべてAzureで行うため、次のセクションでSnowflake側で設定する際に、このApp Registrationにリンクするために必要な情報を得ることができます。

1. [Microsoft Azure Portal](https://portal.azure.com/)に移動して認証します。
2. Azure Entra ID（旧Active Directory）に移動します。
3. **管理**の下にある**アプリの登録**をクリックします。
4. **新規登録**をクリックします。
5. **名前**として`Snowflake GPT OAuth Client`または類似の値を入力します。
6. **サポートされているアカウントの種類**が**単一テナント**に設定されていることを確認します。
7. リダイレクトURIは今のところ無視してください。GPTを設定する際に後で戻ってきます。
8. **登録**をクリックします。
9. **基本情報**の下にある**ディレクトリ（テナント）ID**（`TENANT_ID`）をメモしてください。これを使用して`AZURE_AD_ISSUER`と`AZURE_AD_JWS_KEY_ENDPOINT`を生成します。
    * `AZURE_AD_ISSUER`は`https://sts.windows.net/TENANT_ID/`です
    * `AZURE_AD_JWS_KEY_ENDPOINT`は`https://login.microsoftonline.com/TENANT_ID/discovery/v2.0/keys`です
10. **概要**インターフェースで**エンドポイント**をクリックします。
11. 右側で、**OAuth 2.0認証エンドポイント（v2）**を`AZURE_AD_OAUTH_AUTHORIZATION_ENDPOINT`として、**OAuth 2.0トークンエンドポイント（v2）**を`AZURE_AD_OAUTH_TOKEN_ENDPOINT`としてメモします。
    * エンドポイントは`https://login.microsoftonline.com/90288a9b-97df-4c6d-b025-95713f21cef9/oauth2/v2.0/authorization`と`https://login.microsoftonline.com/90288a9b-97df-4c6d-b025-95713f21cef9/oauth2/v2.0/token`のようになります。
12. **管理**の下にある**APIの公開**をクリックします。
13. **アプリケーションID URI**の横にある**設定**リンクをクリックして、`Application ID URI`を設定します。
    * `Application ID URI`は、`https://your.company.com/4d2a8c2b-a5f4-4b86-93ca-294185f45f2e`のように、組織のディレクトリ内で一意である必要があります。この値は、後続の設定手順で`<SNOWFLAKE_APPLICATION_ID_URI>`として参照されます。
14. プログラマティッククライアントがユーザーの代理として動作するOAuthフローでSnowflakeロールをOAuthスコープとして追加するには、**スコープの追加**をクリックしてSnowflakeロールを表すスコープを追加します。
    * Snowflakeロール名に`session:scope:`プレフィックスを付けてスコープを入力します。例えば、Snowflake Analystロールの場合、`session:scope:analyst`と入力します。
    * 同意できるユーザーを選択します。
    * スコープの**表示名**を入力します（例：Account Admin）。
    * スコープの**説明**を入力します（例：Snowflakeアカウントを管理できます）。
    * **スコープの追加**をクリックします。
    * スコープを`AZURE_AD_SCOPE`として保存します。これは`Application ID URI`と`Scope name`の連結である必要があります。
15. **概要**セクションで、**アプリケーション（クライアント）ID**フィールドから`ClientID`をコピーします。これは以下の手順で`OAUTH_CLIENT_ID`として知られます。
16. **証明書とシークレット**をクリックし、次に**新しいクライアントシークレット**をクリックします。
17. シークレットの説明を追加します。
18. **730日（24か月）**を選択します。テスト目的では、すぐに期限切れにならないシークレットを選択してください。
19. **追加**をクリックします。シークレットをコピーします。これは以下の手順で`OAUTH_CLIENT_SECRET`として知られます。
20. ユーザーの代理でアクセストークンを要求するプログラマティッククライアントの場合、アプリケーションの委任されたアクセス許可を次のように設定します。
    * **APIのアクセス許可**をクリックします。
    * **アクセス許可の追加**をクリックします。
    * **自分のAPI**をクリックします。
    * [Azure ADでOAuthリソースを設定する](https://docs.snowflake.com/en/user-guide/oauth-azure#configure-the-oauth-resource-in-azure-ad)で作成した**Snowflake OAuthリソース**をクリックします。
    * **委任されたアクセス許可**ボックスをクリックします。
    * このクライアントに付与したいアプリケーションで定義されたスコープに関連するアクセス許可をチェックします。
    * **アクセス許可の追加**をクリックします。
    * **管理者の同意を付与**ボタンをクリックして、クライアントにアクセス許可を付与します。テスト目的では、アクセス許可はこの方法で設定されることに注意してください。ただし、本番環境では、この方法でアクセス許可を付与することは推奨されません。
    * **はい**をクリックします。

## Snowflakeでセキュリティ統合を作成する

Azure Entra IDでのアプリ登録が完了したら、次のステップは、そのアプリ登録をExternal OAuth Security Integrationを介してSnowflakeにリンクすることです。セキュリティ統合の`external_oauth_audience_list`パラメータは、Azure Entra IDの設定時に指定した**Application ID URI**と一致する必要があります。

**Issuer**と**JWS Keys endpoint**も、前のステップで収集した値から取得されます。**User Mapping Attribute**は`EMAIL_ADDRESS`または`LOGIN_NAME`のいずれかに設定でき、これによりユーザーのMicrosoftログイン認証情報がSnowflakeのユーザーにマッピングされ、ChatGPTに発行されたアクセストークンによってSnowflakeの権限が確実に尊重されます。

In [ ]:
CREATE OR REPLACE SECURITY INTEGRATION AZURE_OAUTH_INTEGRATION
  TYPE = EXTERNAL_OAUTH
  ENABLED = TRUE
  EXTERNAL_OAUTH_TYPE = 'AZURE'
  EXTERNAL_OAUTH_ISSUER = '<AZURE_AD_ISSUER>'
  EXTERNAL_OAUTH_JWS_KEYS_URL = '<AZURE_AD_JWS_KEY_ENDPOINT>'
  EXTERNAL_OAUTH_AUDIENCE_LIST = ('<SNOWFLAKE_APPLICATION_ID_URI>')
  EXTERNAL_OAUTH_TOKEN_USER_MAPPING_CLAIM = 'upn'
  EXTERNAL_OAUTH_SNOWFLAKE_USER_MAPPING_ATTRIBUTE = 'EMAIL_ADDRESS';

### ミドルウェア情報：

Azure環境で以下の手順を実行することを確認してください：

* Azure Function AppsとAzure Entra App Registrationsを作成するアクセス権限を持つAzure PortalまたはVS Code
* このガイドには、Snowflakeからのレスポンスをラップしてクエリ結果をCSVとしてChatGPTに返すために必要な関数のデプロイと設計に関する[詳細なセクション](#azure-function-app)があります。Azure Function Appにより、ChatGPTはapplication/jsonペイロードよりもファイルレスポンスからより多くのデータを取り込むことができるため、GPTはより大きなデータセットを取り込むことができます。さらに、これらのデータセットは、CSVファイルとしてフォーマットされたレスポンスでのみData Analysis（別名Code Interpreter）で利用可能になります。

### Azure Function App

GPTを作成し、Azure/Snowflake認証を処理したので、次はAzure Function App自体を作成して、SQLクエリを実行し、レスポンスの書式設定を処理できるようにします。これにより、GPTがデータ分析で使用するために結果をCSVとしてダウンロードできるようになります。

Azure Function Appのデプロイに関する詳細については、この[Azure Cookbook Guide](https://cookbook.openai.com/examples/azure/functions)に従ってください。以下に、関数に追加するサンプルコードを示します。

このコードは方向性を示すものです。そのまま動作するはずですが、あなたのGPTの特定のニーズやIT環境に基づいてカスタマイズする必要があります。

### アプリケーションコード

Azure Function Appで以下のフローを設定する必要があります：

* HTTPリクエストからトークンを抽出し、それを使用してSnowflakeに接続する
* SQLクエリを実行し、結果をCSVに書き込む
* そのCSVをBlob Storageに一時的に保存する*
* そのCSVに安全にアクセスするための事前署名URLを生成する*
* openaiFileResponseで応答する

*これらの手順は、GPTにファイルを返すために[url](https://platform.openai.com/docs/actions/getting-started/url-option)オプションではなく[file stream](https://platform.openai.com/docs/actions/getting-started/inline-option)オプションを使用する場合は必要ない可能性があります。詳細は以下で説明します。

必要なライブラリがインストールされ、スクリプトにインポートされていることを確認してください。Python標準ライブラリに加えて、このサンプルスクリプトでは以下を活用しています：

In [ ]:
import azure.functions as func
from azure.storage.blob import BlobServiceClient, generate_blob_sas, BlobSasPermissions, ContentSettings
import snowflake.connector
import jwt    # pyjwt for token decoding

#### Snowflakeへの接続

Snowflakeに接続するには、AuthorizationヘッダーからAzure Entra IDから割り当てられたアクセストークンを抽出し、Snowflakeサーバーに接続する際にそのトークンを使用する必要があります。

この例では、Snowflakeのユーザーネームはメールアドレスであるため、HTTPアクセストークンから抽出されたEntra IDユーザーを、接続に必要なSnowflakeユーザーIDにマッピングすることが簡単になります。組織でこれが当てはまらない場合は、Pythonアプリケーション内でメールアドレスをSnowflakeユーザーIDにマッピングすることができます。

私のアプリケーションは、単一のSnowflakeアカウント（例：ab12345.eastus2.azure）とWarehouseとのインターフェースを構築するように作られています。複数のアカウントやウェアハウスにアクセスする必要がある場合は、GPTアクションパラメータでこれらのパラメータを渡すことを検討し、HTTPリクエストから抽出できるようにすることをお勧めします。

In [ ]:
# Extract the token from the Authorization header
auth_header = req.headers.get('Authorization')
token_type, token = auth_header.split()

try:
    # Extract email address from token to use for Snowflake user mapping
    # If Snowflake usernames are not emails, then identify the username accordingly
    decoded_token = jwt.decode(token, options={"verify_signature": False})
    email = decoded_token.get('upn') 
    
    conn = snowflake.connector.connect(
        user=email, # Snowflake username, i.e., user's email in my example
        account=SNOWFLAKE_ACCOUNT, # Snowflake account, i.e., ab12345.eastus2.azure
        authenticator="oauth",
        token=token,
        warehouse=SNOWFLAKE_WAREHOUSE # Replace with Snowflake warehouse
    )
    logging.info("Successfully connected to Snowflake.")
except Exception as e:
    logging.error(f"Failed to connect to Snowflake: {e}")

#### クエリを実行してCSVを保存

Snowflakeに接続したら、クエリを実行して結果をCSVに保存する必要があります。Snowflakeのロールは有害なクエリの実行を防ぐはずですが、他のプログラマティックなSQLクエリ実行と同様に、アプリケーション内でクエリをサニタイズすることを検討した方が良いでしょう（以下のコードには含まれていません）。

In [ ]:
# Extract SQL query from request parameters or body
sql_query = req.params.get('sql_query')

try:
    # Use the specified warehouse
    cursor = conn.cursor()

    # Execute the query
    cursor.execute(sql_query)
    results = cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description]
    logger.info(f"Query executed successfully: {sql_query}")

    # Convert results to CSV
    csv_file_path = write_results_to_csv(results, column_names)
except Exception as e:
    logger.error(f"Error executing query or processing data: {e}")


def write_results_to_csv(results, column_names):
    try:
        # Create a temporary file
        temp_file = tempfile.NamedTemporaryFile(delete=False, mode='w', newline='')
        csv_writer = csv.writer(temp_file)
        csv_writer.writerow(column_names)  # Write the column headers
        csv_writer.writerows(results)      # Write the data rows
        temp_file.close()  # Close the file to flush the contents
        return temp_file.name  # Return file path
    except Exception as e:
        logger.error(f"Error writing results to CSV: {e}")

#### Blob Storageへのファイル保存

ChatGPTで処理するためにファイルを返す方法は2つあります。[ストリーミング](https://platform.openai.com/docs/actions/getting-started/inline-option)でbase64エンコードされたデータをmimeTypeとファイル名と共にopenaiFileResponseリストレスポンスで送信するか、[URLのリスト](https://platform.openai.com/docs/actions/getting-started/url-option)を返すかのいずれかです。このソリューションでは後者に焦点を当てます。

これを行うには、CSVをAzure Blob Storageにアップロードし、ChatGPTでそのファイルに安全にアクセスするための事前署名URLを返す必要があります。ChatGPTでURLをダウンロードするためには、以下の例のように、そのURLにcontent_typeとcontent_dispositionが含まれている必要があることに注意することが重要です。URLに必要なヘッダーが含まれているかどうかを確認したい場合は、任意のターミナルから`curl -I <url>`を使用できます。

[こちら](https://learn.microsoft.com/en-us/azure/storage/common/storage-configure-connection-string)の手順に従って、Azure storageバケットの接続文字列を取得する必要があります。

In [ ]:
def upload_csv_to_azure(file_path, container_name, blob_name, connect_str):
    try:
        # Create the BlobServiceClient object which will be used to create a container client
        blob_service_client = BlobServiceClient.from_connection_string(connect_str)
        
        # Create a blob client using the local file name as the name for the blob
        blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

        # Upload the file with specified content settings
        with open(file_path, "rb") as data:
            blob_client.upload_blob(data, overwrite=True, content_settings=ContentSettings(
                content_type='text/csv',
                content_disposition=f'attachment; filename="{blob_name}"'
            ))
        logger.info(f"Successfully uploaded {file_path} to {container_name}/{blob_name}")

        # Generate a SAS token for the blob
        sas_token = generate_blob_sas(
            account_name=blob_service_client.account_name,
            container_name=container_name,
            blob_name=blob_name,
            account_key=blob_service_client.credential.account_key,
            permission=BlobSasPermissions(read=True),
            expiry=datetime.datetime.utcnow() + datetime.timedelta(hours=1)  # Token valid for 1 hour
        )

        # Generate a presigned URL using the SAS token
        url = f"https://{blob_service_client.account_name}.blob.core.windows.net/{container_name}/{blob_name}?{sas_token}"
        logger.info(f"Generated presigned URL: {url}")

        return url
    except Exception as e:
        logger.error(f"Error uploading file to Azure Blob Storage: {e}")
        raise

#### openaiFileResponseのフォーマット

最後に、ChatGPTがその応答をファイルまたは一連のファイルとして処理するよう指示するために、応答を適切にフォーマットする必要があります。`openaiFileResponse`は最大10個のURL（または[インラインオプション](https://platform.openai.com/docs/actions/getting-started/inline-option)を使用する場合はbase64エンコーディング）を含むことができるリストです。

In [ ]:
# Format the response so ChatGPT treats it as a file
response = {
    'openaiFileResponse': [csv_url]
}
cursor.close()
conn.close()
return func.HttpResponse(
    json.dumps(response), 
    status_code=200
)

このアプリケーションには多くの動的な要素があるため、Azure Function Appのテストは重要です。ChatGPTは、リクエストとレスポンスがデバッグに必要以上に不透明になることがあるため、テストの場としては困難な場合があります。より制御された環境からHTTPリクエストを呼び出すために、cURLやPostmanを使用してアプリケーションの初期テストを行うことで、問題のデバッグやトリアージをより簡単に行うことができます。これらのツールで期待通りのレスポンスが返されることを確認できたら、GPTの構築に進む準備が整います。

## ChatGPTステップ

### カスタムGPT指示

Custom GPTを作成したら、以下のテキストをInstructionsパネルで参考として使用してください。質問がありますか？この手順の詳細については、[Getting Started Example](https://platform.openai.com/docs/actions/getting-started)をご確認ください。

申し訳ございませんが、翻訳すべきテキストが「##### Example Instructions」という見出しのみで、実際の内容が含まれていないようです。

この見出しを翻訳すると以下のようになります：

##### 使用例の手順

もし他に翻訳したい内容がございましたら、お知らせください。

ChatGPTがSQLクエリを適切に作成するためには、テーブルスキーマを理解することが重要です。これを行う方法はいくつかありますが、この手順書は最も直接的な方法を示しています。複数の異なるテーブル、スキーマ、データベースを扱ったり、時間の経過とともに変化する傾向があるスキーマを動的に学習したりできる、さまざまなバージョンのSnowflake GPTを構築するための追加手順を公開する予定です。

以下は、単一のスキーマとテーブルを扱う際の基本的な手順です。このGPTは単一のユースケース（2013年1月のNYCからのフライトデータの分析）に最適化されており、最もシンプルな手順で最も信頼性の高いGPTパフォーマンスを提供できます。

あなたはSnowflakeからデータを取得するSQLクエリの作成エキスパートです。ユーザーのプロンプトをSQLクエリに変換することを支援します。フライトデータに関するあらゆる質問は、テーブル`FLIGHTS.PUBLIC.JAN_2013_NYC`にアクセスするSnowflake SQLクエリに変換されます。すべてのクエリは"sql_query"パラメータに渡してください。

テーブルのスキーマには以下が含まれます：

```
ID	NUMBER	各フライトの一意識別子
YEAR	NUMBER	フライトの年
MONTH	NUMBER	フライトの月
DAY		NUMBER	フライトが出発した月の日
DEP_TIME	NUMBER	フライトの実際の出発時刻
SCHED_DEP_TIME	NUMBER	フライトの予定出発時刻
DEP_DELAY	NUMBER	出発遅延（分）（負の値は早期出発を示す）
ARR_TIME	NUMBER	フライトの実際の到着時刻
SCHED_ARR_TIME	NUMBER	フライトの予定到着時刻
ARR_DELAY	NUMBER	到着遅延（分）（負の値は早期到着を示す）
CARRIER_CODE	TEXT	航空会社のキャリアコード
FLIGHT	NUMBER	フライト番号
TAILNUM	TEXT	航空機の機体番号
ORIGIN_AIRPORT_CODE	TEXT	出発空港コード
DEST_AIRPORT_CODE	TEXT	到着空港コード
AIR_TIME	NUMBER	フライトの総飛行時間（分）
DISTANCE	NUMBER	フライトが移動した距離（マイル）
HOUR	NUMBER	予定出発時刻の時間部分
MINUTE	NUMBER	予定出発時刻の分部分
TIME_HOUR	NUMBER	フライトが出発した時刻（最も近い時間に四捨五入）
CARRIER_NAME	TEXT	航空会社キャリアのフルネーム
ORIGIN_AIRPORT_NAME	TEXT	出発空港のフルネーム
ORIGIN_REGION	TEXT	出発空港の地域コード
ORIGIN_MUNICIPALITY	TEXT	出発空港が所在する都市
ORIGIN_COORDINATES	TEXT	出発空港の地理座標
DEST_AIRPORT_NAME	TEXT	到着空港のフルネーム
DEST_REGION	TEXT	到着空港の地域コード
DEST_MUNICIPALITY	TEXT	到着空港が所在する都市
DEST_COORDINATES	TEXT	到着空港の地理座標
```

ユーザーがフライトに関するデータを求めた場合、以下を実行してください：

1. `executeSQL`アクションを使用してAzure関数エンドポイントにPOSTリクエストを送信する
2. アクションレスポンスの一部として返されるファイルを受信する。それをスプレッドシートとして表示する
3. ファイルに対して分析を実行し、ユーザーが求めた必要な情報を提供する

ユーザーはcode interpreterでデータについて質問したいと考えているため、取得したデータセットからのデータ分析の洞察にはそれを使用してください。

### OpenAPI スキーマ

Custom GPTを作成したら、以下のテキストをActionsパネルにコピーし、プレースホルダーの値をあなたの具体的な関数の詳細に置き換え、Azure Function Appに組み込んだ追加の入力に基づいてパラメータを更新してください。

質問がありますか？この手順の詳細については、[Getting Started Example](https://platform.openai.com/docs/actions/getting-started)をご確認ください。

In [ ]:
openapi: 3.1.0
info:
  title: Snowflake GPT API
  description: API to execute SQL queries on Snowflake and get the results as a CSV file URL.
  version: 1.0.0
servers:
  - url: https://<server-name>.azurewebsites.net
    description: Azure Function App server running Snowflake integration application
paths:
  /api/<function_name>?code=<code>:
    post:
      operationId: executeSQL
      summary: Executes a SQL query on Snowflake and returns the result file URL as a CSV.
      requestBody:
        required: true
        content:
          application/json:
            schema:
              type: object
              properties:
                sql_query:
                  type: string
                  description: The SQL query to be executed on Snowflake.
              required:
                - sql_query
      responses:
        '200':
          description: Successfully executed the query.
          content:
            application/json:
              schema:
                type: object
                properties:
                  openaiFileResponse:
                    type: array
                    items:
                      type: string
                      format: uri
                    description: Array of URLs pointing to the result files.
        '401':
          description: Unauthorized. Missing or invalid authentication token.
        '400':
          description: Bad Request. The request was invalid or cannot be otherwise served.
        '500':
          description: Internal Server Error. An error occurred on the server.
components:
  schemas: {} 

### FAQ & トラブルシューティング

* ChatGPTに返されるファイルのサイズは10MBに制限されています。返されるファイルがそれより大きい場合、リクエストが失敗する可能性があります。これらの制限に遭遇した場合は、SQLコマンドにLIMITを含めるようにしてください。
* _そもそもなぜAzure Function Appが必要なのか？_ ChatGPTのデータ分析機能（別名Code Interpreter）は、モデルのコンテキストウィンドウとは分離された安全なPython環境に依存しています。データ分析に渡されるデータは、現在ファイルをアップロードすることで行う必要があります。データを返すGPTアクションは、そのデータをCSVまたは他のデータファイル形式として返す必要があります。GPTアクション経由でファイルを返すには、レスポンスを`openaiFileResponse`オブジェクトでラップする必要があります。これには、レスポンスを適切にフォーマットするためのカスタムコードが必要です。
* _私の会社はAzure以外のクラウドプロバイダーを使用しています。_ GPTアクション経由で他のミドルウェア関数をChatGPTに接続する場合は、他の[AWS](https://cookbook.openai.com/examples/chatgpt/gpt_actions_library/gpt_middleware_aws_function)または[GCP](https://cookbook.openai.com/examples/chatgpt/gpt_actions_library/gpt_middleware_google_cloud_function)ミドルウェアクックブックを参照してください。このクックブックで説明されている概念を使用して、ミドルウェアアプリを構築する際の考慮事項についてアドバイスできますが、そのミドルウェアをSnowflakeに接続することは、異なるクラウドプロバイダーでは異なる場合があります。例えば、SnowflakeはAzure Entra IDとの連携専用のExternal OAuth統合を構築しています。
* _GPTがアクセスできるデータセットを制限するにはどうすればよいですか？_ ChatGPTがSnowflake内で持つアクセス範囲を制限することは重要です。これを行う方法はいくつかあります：
    * Snowflakeロールは、誰がどのテーブルにアクセスできるかを制限でき、Azure Entra IDによってプロビジョニングされたGPTユーザーのアクセストークンによって尊重されます
    * ミドルウェア関数で、アクセスされるテーブルがそのアプリケーションで承認されているかを確認するサニティチェックを追加できます
    * PIIなどの機密情報を除去した、ChatGPTとの統合専用の全く新しいDatabase/Warehouseを生成することを検討してください
* _スキーマが間違ったウェアハウスやデータセットを呼び出す：_ ChatGPTが間違ったウェアハウスやデータベースを呼び出す場合は、(a)どのウェアハウス/データベースを呼び出すべきかをより明確にするか、(b)クエリを実行する前にユーザーにそれらの正確な詳細を提供するよう要求するように、指示を更新することを検討してください

_優先的に対応してほしい統合機能はありますか？統合機能にエラーがありますか？GitHubでPRやissueを作成していただければ、確認いたします。_